
# 04 · Umfrage-Ingest (Stichprobe 2025)

Dieses Notebook lädt die **Stichprobe (umfrage_2025.xlsx)**, erkennt das Format (Single-Choice vs. Multi-Choice),
harmonisiert die Kategorien gemäß `mapping_statista_to_kanon.csv` (aus Notebook 01), und exportiert die Stichprobe als
**long**- und **wide**-Format für den Vergleich.

**Erwartung an die Eingabedatei** `data/raw/umfrage_2025.xlsx`:
- *Single-Choice*: eine Spalte mit der gewählten Kategorie (z. B. `Kategorie`/`category`/`Warengruppe`/`Produktkategorie`/`Bereich`)
- *Multi-Choice*: mehrere Spalten (je Kategorie) mit 0/1, False/True oder Ja/Nein

Wenn dein Excel anders aufgebaut ist, passe unten `CATEGORY_COL` oder `MULTI_COLS` an.


In [18]:

# 04_umfrage_ingest — Cell 1: Imports, Pfade, Mapping laden
from __future__ import annotations

from pathlib import Path
import json, re
import pandas as pd

# Projektpfade
NB_DIR  = Path.cwd().resolve()
BASE    = NB_DIR.parents[0] if NB_DIR.name.lower() == "notebooks" else NB_DIR
DATA    = BASE / "data"
RAW     = DATA / "raw"
OUT     = DATA / "processed"
OUT.mkdir(parents=True, exist_ok=True)

# Config & Mapping (aus 01)
CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]
MAP_CSV = Path(CONFIG["paths"]["mapping_csv"])
mapping_df = pd.read_csv(MAP_CSV)

# Mapping-Lookup (normalisiert -> KANON)
def norm_text(s: str) -> str:
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = s.replace("&", "und").replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    s = s.replace("accressoires", "accessoires").replace("hi tech", "high tech")
    return s
MAP_NORM = dict(zip(mapping_df["source_normalized"].map(str), mapping_df["kanon"]))


In [19]:
# 04_umfrage_ingest — Cell 2 (robust, 3 Modi)
XLSX_PATH = RAW / "umfrage_2025.xlsx"  # bitte Datei hier ablegen
assert XLSX_PATH.exists(), f"Eingabedatei fehlt: {XLSX_PATH}"

import pandas as pd, re
from difflib import SequenceMatcher

# Excel einlesen (erste Tabelle)
xls = pd.read_excel(XLSX_PATH, sheet_name=0)
print("Excel geladen:", XLSX_PATH, "| Shape:", xls.shape)
display(xls.head(3))

# --- Helfer ---
BIN_TRUE  = {"1","true","yes","ja","x","wahr","checked"}
BIN_FALSE = {"0","false","no","nein","falsch","","unchecked","na"}

def looks_binary(series: pd.Series) -> bool:
    s = series.dropna()
    if s.empty:
        return False
    # wenige Ausprägungen -> binär
    if s.nunique() <= 2:
        return True
    v = s.astype(str).str.strip().str.lower()
    frac_known = v.isin(BIN_TRUE | BIN_FALSE).mean()
    return frac_known > 0.9

def header_to_kanon(header: str) -> str | None:
    """Fuzzy-Header-Mapping: exact, contains, ratio."""
    nrm = norm_text(header)
    # 1) exaktes Normalisat im Mapping?
    if nrm in MAP_NORM:
        return MAP_NORM[nrm]
    # 2) Teilstring irgendeines Mapping-Keys?
    for src_norm, tgt in MAP_NORM.items():
        if src_norm and src_norm in nrm:
            return tgt
    # 3) KANON enthalten?
    for k in KANON:
        if norm_text(k) in nrm:
            return k
    # 4) Fuzzy Ratio gg. KANON
    best, best_k = 0.0, None
    for k in KANON:
        r = SequenceMatcher(None, norm_text(k), nrm).ratio()
        if r > best:
            best, best_k = r, k
    return best_k if best >= 0.6 else None

def token_to_kanon(tok: str) -> str | None:
    """Einzelnes Token (bei multi_delimited) → KANON (fuzzy)."""
    t = re.sub(r"[\"'\\[\\](){}]", "", str(tok)).strip()
    if not t:
        return None
    nrm = norm_text(t)
    if nrm in MAP_NORM:
        return MAP_NORM[nrm]
    # Teilstring
    for src_norm, tgt in MAP_NORM.items():
        if src_norm and src_norm in nrm:
            return tgt
    # Fuzzy gg. KANON
    best, best_k = 0.0, None
    for k in KANON:
        r = SequenceMatcher(None, norm_text(k), nrm).ratio()
        if r > best:
            best, best_k = r, k
    return best_k if best >= 0.6 else None

def looks_delimited(series: pd.Series) -> bool:
    """Erkennt Spalte mit Mehrfachauswahl (Komma/Semikolon/Pipe/Slash)."""
    s = series.dropna().astype(str)
    if s.empty:
        return False
    # Anteil Zeilen mit Trennzeichen
    has_delim = s.str.contains(r"[;,|/]", regex=True).mean()
    return has_delim >= 0.2

# --- Manuelle Overrides (falls bekannt) ---
CATEGORY_COL = None   # z. B. "Kategorie"
MULTI_COLS   = []     # z. B. ["Bekleidung", "Schuhe", ...]
DELIMITED_COL = None  # z. B. "Ausgewählte Kategorien"
DELIMS = r"[;,|/]"    # erlaubte Trennzeichen in multi_delimited

# --- Auto-Detection nur, wenn nichts manuell gesetzt ist ---
mode = None
if CATEGORY_COL is None and not MULTI_COLS and DELIMITED_COL is None:
    # (A) Single-Choice Kandidaten: Spalte, deren Werte gut ins Mapping passen
    best_col, best_rate = None, -1.0
    for col in xls.columns:
        s = xls[col].dropna().astype(str).head(500)
        if s.empty:
            continue
        mapped = s.map(lambda x: MAP_NORM.get(norm_text(x)))
        rate = mapped.notna().mean()
        if rate > best_rate:
            best_col, best_rate = col, rate
    if best_rate >= 0.6:
        CATEGORY_COL = best_col
        mode = "single"

    # (B) Multi-Wide (Checkbox-Matrix)
    if mode is None:
        cand = []
        for col in xls.columns:
            if looks_binary(xls[col]):
                tgt = header_to_kanon(str(col))
                if tgt:
                    cand.append((col, tgt))
        if len(cand) >= 1:
            MULTI_COLS = [c for c, _ in cand]
            mode = "multi_wide"

    # (C) Multi-Delimited (eine Textspalte, mehrere Kategorien)
    if mode is None:
        best_col, best_tokrate = None, -1.0
        for col in xls.columns:
            if xls[col].dtype == object and looks_delimited(xls[col]):
                sample = xls[col].dropna().astype(str).head(200)
                # Tokenisieren & Maprate prüfen
                tokens = []
                for s in sample:
                    tokens.extend(re.split(DELIMS, s))
                if not tokens:
                    continue
                mrate = pd.Series(tokens).map(token_to_kanon).notna().mean()
                if mrate > best_tokrate:
                    best_col, best_tokrate = col, mrate
        if best_col is not None and best_tokrate >= 0.5:
            DELIMITED_COL = best_col
            mode = "multi_delimited"

# Modus final bestimmen (falls Override gesetzt)
if mode is None:
    if CATEGORY_COL is not None: mode = "single"
    elif DELIMITED_COL is not None: mode = "multi_delimited"
    elif MULTI_COLS: mode = "multi_wide"
    else: mode = "unknown"

print(f"Erkannter Modus: {mode}")
if mode == "single":
    print("Single-Choice Spalte:", CATEGORY_COL)
elif mode == "multi_wide":
    print("Multi-Choice Spalten (heuristisch):", MULTI_COLS[:20], "...")
elif mode == "multi_delimited":
    print("Multi-Delimited Spalte:", DELIMITED_COL, "| Delimiter:", DELIMS)
else:
    print("Konnte das Format nicht erkennen. Bitte CATEGORY_COL / MULTI_COLS / DELIMITED_COL manuell setzen.")
    raise AssertionError("Format unbekannt → Bitte in Cell 2 manuell konfigurieren.")


Excel geladen: D:\Q3_2025\data-analytics\project\data\raw\umfrage_2025.xlsx | Shape: (39, 43)


,ID,Startzeit,Fertigstellungszeit,E-Mail,Name,Zeitpunkt der letzten Änderung,Wie alt sind Sie?,Welchem Geschlecht ordnen Sie sich zu?,Wie häufig kaufen Sie online ein?,Was kaufen Sie derzeit regelmäßig online ein?,...,Sprachassistenten / Smart-Geräte2,COVID-19-Pandemie,Nachhaltigkeit,Digitalisierung (z. B. Apps),Inflation / Preissteigerungen,Welcher dieser Faktoren wird Ihr zukünftiges Online-Kaufverhalten voraussichtlich am stärksten beeinflussen?,Wie hat sich Ihr durchschnittlicher Einkaufswert bei Online-Bestellungen in den letzten fünf Jahren verändert?,"Für welche dieser Aspekte sind Sie heute eher bereit, einen höheren Preis zu zahlen?",Welche der folgenden Zahlungsarten nutzen Sie beim Online-Kauf regelmäßig?,Wie hat sich Ihre Nutzung von „Buy Now Pay Later“-Angeboten (z. B. Klarna) in den letzten Jahren verändert?
0,1,2025-07-25 16:13:32,2025-07-25 16:16:53,anonymous,NaN,NaN,18-24,Weiblich,Wöchentlich,Kleidung / Schuhe;Möbel / Wohnaccessoires;,...,Nie genutzt,Eher stark,Neutral,Eher stark,Neutral,Bequemlichkeit / Zeitersparnis;,Deutlich gestiegen,Höhere Produktqualität / Markenprodukte;Lokale...,PayPal;Sofortüberweisung;,Ich habe BNPL noch nie genutzt
1,2,2025-07-25 16:13:30,2025-07-25 16:17:32,anonymous,NaN,NaN,35-44,Weiblich,Mehrmals im Monat,"Elektronik (z. B. Smartphones, Haushaltsgeräte...",...,Nie genutzt,Eher stark,Gar nicht,Eher stark,Eher stark,Bequemlichkeit / Zeitersparnis;,Gleich geblieben,Verbesserter Kundenservice / Rückgabeservice;,PayPal;Kreditkarte;,Ich habe BNPL noch nie genutzt
2,3,2025-07-25 16:13:16,2025-07-25 16:18:22,anonymous,NaN,NaN,18-24,Männlich,Mehrmals im Monat,"Elektronik (z. B. Smartphones, Haushaltsgeräte...",...,Nie genutzt,Gar nicht,Eher stark,Eher stark,Neutral,Preisentwicklung / Inflation;Nachhaltigkeit;Be...,Deutlich gestiegen,Nachhaltige oder umweltfreundliche Produkte;Hö...,PayPal;Apple Pay;,Ich habe BNPL noch nie genutzt


Erkannter Modus: multi_delimited
Multi-Delimited Spalte: Was kaufen Sie derzeit regelmäßig online ein? | Delimiter: [;,|/]


In [20]:
# 04_umfrage_ingest — Cell 3 (verarbeitet single / multi_wide / multi_delimited)
from collections import defaultdict
import numpy as np

def map_to_kanon_single(series: pd.Series) -> pd.Series:
    return series.map(lambda x: MAP_NORM.get(norm_text(x), "IGNORE"))

def to_bool(x) -> bool:
    if pd.isna(x): return False
    v = str(x).strip().lower()
    if v in BIN_TRUE:  return True
    if v in BIN_FALSE: return False
    try:
        return float(v) > 0
    except:
        return False

if mode == "single":
    ser = xls[CATEGORY_COL].astype(str)
    ser_mapped = map_to_kanon_single(ser)
    df = (ser_mapped.to_frame("Kategorie")
                    .assign(sel=True)
                    .query("Kategorie != 'IGNORE'")
                    .groupby("Kategorie", as_index=False)["sel"].sum()
                    .rename(columns={"sel":"cnt"}))
    N = len(xls)

elif mode == "multi_wide":
    sub = xls[MULTI_COLS].copy()
    # Bool konvertieren
    for c in sub.columns:
        sub[c] = sub[c].map(to_bool)
    # Header → KANON (fuzzy)
    col2kanon = {c: header_to_kanon(str(c)) for c in sub.columns}
    # Spalten ohne Mapping verwerfen
    sub = sub[[c for c in sub.columns if col2kanon[c]]]
    # ANY pro KANON (mind. 1 Spalte der Gruppe True)
    groups = defaultdict(list)
    for c, tgt in col2kanon.items():
        groups[tgt].append(c)
    any_df = pd.DataFrame({tgt: sub[cols].any(axis=1) for tgt, cols in groups.items()})
    N = len(any_df)
    cnt_series = any_df.sum(axis=0).rename("cnt")
    df = cnt_series.reset_index().rename(columns={"index":"Kategorie"})

elif mode == "multi_delimited":
    col = DELIMITED_COL
    # Tokenisierung, Mapping je Zeile → Set (keine Doppelzählung pro Respondent/Kategorie)
    sel = []
    for s in xls[col].fillna("").astype(str):
        toks = [t.strip() for t in re.split(DELIMS, s) if t.strip()]
        mapped = {token_to_kanon(t) for t in toks}
        mapped = {m for m in mapped if m is not None}
        sel.append(mapped)
    N = len(sel)
    # Count: Anzahl Respondenten, die Kategorie mind. 1x nannten
    cnt = {k: 0 for k in KANON}
    for s in sel:
        for k in s:
            if k in cnt:
                cnt[k] += 1
    df = pd.DataFrame({"Kategorie": list(cnt.keys()), "cnt": list(cnt.values())})

else:
    raise RuntimeError("Unbekannter Modus – Verarbeitung abgebrochen.")

# auf KANON komplettieren
df = df.set_index("Kategorie").reindex(KANON).fillna({"cnt":0}).reset_index()
df["share_%"] = 100 * df["cnt"].astype(float) / float(N if N else 1)
df["metric"]  = "share_of_respondents"
df["year"]    = 2025

display(df.sort_values("share_%", ascending=False).head(10))
print(f"Stichprobengröße (Respondenten N) = {int(N)}")


,Kategorie,cnt,share_%,metric,year
0,Kleidung / Schuhe,27,69.230769,share_of_respondents,2025
5,Hobby- & Freizeitartikel,21,53.846154,share_of_respondents,2025
3,Bücher / Medien / Software,18,46.153846,share_of_respondents,2025
1,"Elektronik (z. B. Smartphones, Haushaltsgeräte)",17,43.589744,share_of_respondents,2025
4,Medikamente / Drogerieartikel,11,28.205128,share_of_respondents,2025
2,Lebensmittel / Getränke,6,15.384615,share_of_respondents,2025
6,Möbel / Wohnaccessoires,0,0.000000,share_of_respondents,2025


Stichprobengröße (Respondenten N) = 39


In [21]:

# 04_umfrage_ingest — Cell 4: Exporte (long & wide)
long_path = OUT / "umfrage_2025_long.csv"
wide_path = OUT / "umfrage_2025_wide.csv"

df_long = df[["year","Kategorie","cnt","share_%","metric"]].sort_values(["year","Kategorie"])
df_wide = df_long.pivot(index="Kategorie", columns="year", values="share_%").fillna(0.0)

df_long.to_csv(long_path, index=False, encoding="utf-8")
df_wide.reset_index().to_csv(wide_path, index=False, encoding="utf-8")

print("Exportiert:")
print("-", long_path)
print("-", wide_path)
print(f"Stichprobengröße (N) = {int(df['cnt'].sum()) if 'cnt' in df else 'n/a'} | Respondenten = {int(len(xls))}")


Exportiert:
- D:\Q3_2025\data-analytics\project\data\processed\umfrage_2025_long.csv
- D:\Q3_2025\data-analytics\project\data\processed\umfrage_2025_wide.csv
Stichprobengröße (N) = 100 | Respondenten = 39
